In [14]:
# importing the relevant libraries
!pip install pandas
import pandas as pd
!pip install sqlalchemy
import sqlalchemy

In [15]:
def showAllRows (dataFrame):
    pd.set_option('display.max_rows', None)
    display(dataFrame)
    pd.reset_option('display.max_rows')

### Task 1
The first task is to demonstrate that you have a working database containing the FakeNewsCorpus dataset.<br>
Explain your choice of schema design.<br>
You have been working on this task on a small subset of the data during the TA-sessions.<br>
For this milestone, demonstrate that your database contains a larger number of rows (e.g. one million - or however many you can reasonably work with on your available hardware), and that it supports simple queries.


In [16]:
# define relevant file
dataForMilestone2 = '../250t-raw.csv'


# reading the csv-file
data = pd.read_csv(dataForMilestone2)
data[:]

C:\Users\jeppe\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,NaN,[''],THE UNIVERSE ceases to exist when we are not l...,NaN,NaN,NaN
1,2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN
2,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN
3,5,10,barenakedislam.com,hate,http://barenakedislam.com/2017/12/27/following...,The Central American nation and six other stat...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Following Guatemala’s decision to move its emb...,"Pamela Jeffersons, Cleavis Nowell, Cleavisnowe...",NaN,[''],NaN,NaN,NaN,NaN
4,8,14,blackagendareport.com,unreliable,https://blackagendareport.com/articlelist/inte...,"“When the police finally left the campus, arou...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Black Agenda Report,"Ann Garrison, Bar Contributor, Ken Morgan, Jem...",NaN,[''],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250524,9913,1170100,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976ANK...,Raw content\n\nCONFIDENTIAL PAGE 01 ANKARA 019...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976ANKARA01989,NaN,NaN,[''],NaN,View Tags,NaN,NaN
250525,9916,1170103,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974JAK...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1974JAKART02409,NaN,NaN,[''],NaN,View Tags,NaN,NaN
250526,9923,1170110,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976ROM...,Raw content\n\nLIMITED OFFICIAL USE PAGE 01 RO...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976ROME16372,NaN,NaN,[''],NaN,View Tags,NaN,NaN
250527,9924,1170111,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976STA...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976STATE222598,NaN,NaN,[''],NaN,View Tags,NaN,NaN


In [17]:
def fieldLengthPrinter (string):
  print(column, len(data[string].value_counts()))

for column in data:
  fieldLengthPrinter (column)

Unnamed: 0 20007
id 250513
domain 356
type 12
url 250511
content 178223
scraped_at 2
inserted_at 1
updated_at 1
title 222069
authors 29030
keywords 0
meta_keywords 30093
meta_description 65459
tags 41991
summary 0
source 0


In [18]:
# Main Database
dataMain = data[['id','url','content','title']]
dataMain = dataMain.rename(columns={'id': 'ArticleID'})
dataMain = dataMain.set_index('ArticleID')
# display(dataMain)

# Databases with related fields
dataAuthors = data[['authors']].drop_duplicates().reset_index(drop=True)
dataAuthors.index.name='AuthorsID'
dataType = data[['type']].drop_duplicates().reset_index(drop=True)
dataType.index.name='TypeID'
dataDomain = (data[['domain']].drop_duplicates().reset_index(drop=True))
dataDomain.index.name='DomainID'

display(dataAuthors[:])
# dataType[:]
# dataDomain[:]

dataAuthors.count()

# Databases connecting Articles to Related Databases
# dataArtIdAutId
# dataArtIdTypeId
# dataArtIdDomainId

,authors
AuthorsID,
0,Sean Martin
1,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ..."
2,"Cleavis Nowell, Cleavisnowell, Clarence J. Fei..."
3,"Pamela Jeffersons, Cleavis Nowell, Cleavisnowe..."
4,"Ann Garrison, Bar Contributor, Ken Morgan, Jem..."
...,...
29026,"David Wright, Wayne Rooney"
29027,"Ben Jefferson, Jan Vertonghen"
29028,"Lauren Clark, Sonal Davda, A Pharmacist For Ch..."


authors    29030
dtype: int64

In [37]:
engine = sqlalchemy.create_engine('sqlite://', echo=False)

data.to_sql('raw', con=engine)
dataMain.to_sql('articles', con=engine)
dataAuthors.to_sql('authors', con=engine)
dataType.to_sql('type', con=engine)
dataDomain.to_sql('domains', con=engine)

In [36]:
engine.execute("SELECT url FROM articles").fetchall()
engine.execute("SELECT * FROM authors WHERE authors IS NULL").fetchall()

[(5, None)]

### Task 2.

List the relations you have created in your database. For each relation:

1.   list its attributes
2.   list its functional dependencies.
3.   list all the primary keys.

Is each relation in BCNF form?<br>
If not, show how to transform the tables in BCNF and explain why it might be better (or not) to use the BCNF relations in your database.

### Task 3.

Once your database is loaded, you can start issuing queries to better understand the characteristics of the data.<br>
Formulate the following queries in the database languages requested (in the square brackets following each item) and briefly discuss what you observe when you execute them over your database:   

1.   List the domains of news articles of reliable type and scraped at or after January 15, 2018. NOTE: Do not include duplicate domains in your answer. \[Languages: relational algebra and SQL]
2.   List the name(s) of the most prolific author(s) of news articles of fake type. An author is among the most prolific if it has authored as many or more fake news articles as any other author in the dataset. \[Languages: extended relational algebra and SQL]
3.   Count the pairs of article IDs that exhibit the exact same set of meta-keywords, but only return the pairs where the set of meta-keywords is not empty. \[Language: SQL]

In [62]:
engine.execute("SELECT DISTINCT domain FROM raw WHERE type='reliable' AND scraped_at>'2018-01-15'").fetchall()
# engine.execute("SELECT COUNT(DISTINCT authors) FROM raw WHERE type='fake'").fetchall()

[('christianpost.com',), ('consortiumnews.com',), ('nutritionfacts.org',)]

### Task 4.

Now that we have our data in a database, let’s revisit the “interesting observations” task from Milestone 1 - but now using queries to the database.<br>
The idea is to write database queries (e.g. using GROUP BY and COUNT) that explore features of the data set that are relevant to the fake news prediction task: outliers, artefacts.<br>
It’s OK to investigate the same issues as in Milestone 1 (now using database queries) - but you are also very welcome to come up with completely new queries.<br>
You should write at least 3 such queries.

In [7]:
numOfDomains = len(data['domain'].value_counts())
print("There are",numOfDomains,"domains")

There are 356 domains


In [8]:
numOfTypes = len(data['type'].value_counts())
numOfDomainTypeCombinations = len(dict(zip(data.domain,data.type)))
print("There are %d domains and %d domain-type combinations" %(numOfDomains,numOfDomainTypeCombinations))

There are 356 domains and 357 domain-type combinations


In [9]:
numOfAuthors = len(data['authors'].value_counts())
numOfDomainAuthorsCombinations = len(dict(zip(data.domain,data.authors)))
print("There are %d domains and %d domain-Authors ombinations" % (numOfDomains,numOfDomainAuthorsCombinations))

There are 356 domains and 357 domain-Authors ombinations
